In [1]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from joblib import dump, load

import sys


In [2]:

  # prediction path
INPUT_SPAN_FILE_PATH = "/data/edan/for_edan/cofie_spans.txt"

def embed_text(text, model):
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)  # Batch size 1
    input_ids = input_ids.to('cuda:1')
    outputs = model(input_ids)
    last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
    return last_hidden_states 

model_version = 'allenai/scibert_scivocab_uncased'
do_lower_case = True
model = BertModel.from_pretrained(model_version)
model.to('cuda:1')
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)
input_file = open(INPUT_SPAN_FILE_PATH)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
def get_words(input_file, model):
  word_list = []
  word_em_list = []
  count = 0
  for line in input_file:
    if count% 10000 == 0:
      print(count)
    # if count == 100000:
    #   break
    count += 1
    if line[:-1] not in word_list:
      word_list.append(line[:-1])
      word_em_list.append(embed_text(word_list[-1], model).mean(1).detach().cpu().numpy()[0])
  return word_list, word_em_list

word_list, word_em_list = get_words(input_file, model)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000


In [30]:
# dump(word_list, "wordlist.joblib")
# dump(word_em_list, "wordemlist.joblib")

['wordemlist.joblib']

In [5]:
word_list = load("wordlist.joblib")
word_em_list = load("wordemlist.joblib")

In [6]:
word_em_list[1]

array([-4.74439204e-01, -1.00434303e+00,  1.52064696e-01, -8.43470871e-01,
       -4.33146238e-01, -6.72840297e-01,  7.26036578e-02,  4.77734983e-01,
       -6.03177071e-01,  3.62054515e-03,  1.26820192e-01,  4.32783574e-01,
       -9.95831370e-01, -3.75104219e-01,  1.98212340e-01,  2.25550041e-01,
       -1.42978489e+00, -4.48473126e-01,  1.06704164e+00, -8.14333335e-02,
       -1.36007471e-02,  4.31536883e-02,  7.67221525e-02, -1.59255624e-01,
        1.20235312e+00,  6.01322830e-01,  4.49224621e-01, -4.32977825e-01,
       -2.39725951e-02, -4.39895332e-01, -2.53334522e-01, -2.51923352e-01,
       -1.15095198e+00, -3.58465798e-02, -9.18443501e-02,  1.06256700e+00,
        2.69555837e-01, -7.80462921e-01, -3.57004076e-01, -3.97289008e-01,
       -2.33315185e-01,  2.84131944e-01,  1.80199817e-01,  5.13347030e-01,
       -8.07753026e-01, -3.11689568e-03,  1.00894117e+00, -3.09087545e-01,
       -1.55323163e-01,  3.24112803e-01, -2.03603581e-01, -2.03133196e-01,
        4.66481119e-01,  

In [7]:
kmeans = load("training_kmeans.joblib")
predictions = kmeans.predict(word_em_list)

3


In [16]:
output_file = open("k_means_clusters_predict.tsv", "w")
for word, prediction in zip(word_list, predictions):
    output_file.write(word + "\t" + str(prediction) + "\n")

In [13]:
print(len(word_list))
print(len(word_em_list))
print(len(predictions))

1248011
1248011
1248011
